In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import add_dummy_feature

alt.data_transformers.disable_max_rows()

# Lab 6: Regression

This lab covers the nuts and bolts of fitting linear models using the `sklearn.linear_model` module. The linear model expresses a response variable, $y$, as a linear function of $p - 1$ explanatory variables $x_1, \dots, x_{p - 1}$ and a random error $\epsilon$. Its general form is:

$$y = \beta_0 + \beta_1 x_1 + \cdots + \beta_{p - 1} x_{p - 1} + \epsilon \qquad \epsilon \sim N(0, \sigma^2)$$

Usually, the response and explanatory variables and error term are indexed by observation $i = 1, \dots, n$ so that the model describes a dataset comprising $n$ values of each variable:

$$y_i = \beta_0 + \beta_1 x_{i1} + \cdots + \beta_{p - 1} x_{i, p - 1} + \epsilon_i \qquad\begin{cases} \epsilon_i \sim N(0, \sigma^2) \\ i = 1, \dots, n\end{cases}$$

Because the indices get confusing to keep track of, it is much easier to express the model in matrix form as

$$\mathbf{y} = \mathbf{X}\beta + \epsilon$$

where:

$$\mathbf{y} = \left[\begin{array}{c} y_1 \\ y_2 \\ \vdots \\ y_n \end{array}\right]_{\;n \times 1} \qquad
    \mathbf{X} = \left[\begin{array}{cccc} 
        1 &x_{11} &\cdots &x_{1, p - 1} \\
        1 &x_{21} &\cdots &x_{2, p - 1} \\
        \vdots &\vdots &\ddots &\vdots \\
        1 &x_{n1} &\cdots &x_{n, p - 1}
        \end{array}\right]_{\;n \times p} \qquad
    \beta = \left[\begin{array}{c} \beta_0 \\ \beta_1 \\ \vdots \\ \beta_{p - 1} \end{array} \right]_{\;p \times 1} \qquad
    \epsilon = \left[\begin{array}{c} \epsilon_1 \\ \epsilon_2 \\ \vdots \\ \epsilon_n \end{array}\right]_{\;n \times 1}$$
    

**Fitting** a model of this form means _**estimating the parameters**_ $\beta_0, \beta_1, \dots, \beta_{p - 1}$ and $\sigma^2$ from a set of data. 
* The ordinary least squares estimates of $\beta$, which are best under most circumstances, are
$$\hat{\beta} = (\mathbf{X'X})^{-1}\mathbf{X'y}$$
* The error variance $\sigma^2$ can be estimated by
$$\hat{\sigma}^2 = \frac{1}{n - p - 1}\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)'\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)$$

When fitting a linear model, it is also of interest to quantify uncertainty by estimating the variability of $\hat{\beta}$ and measure overall quality of fit. This lab illustrates that process and the computations involved.

---
### Objectives

In this lab, you'll learn how to use the `sklearn.linear_model` module to:
* compute OLS estimates;
* calculate fitted values and residuals;
* compute the error variance estimate.

In addition, you'll see how to calculate:
* the variance-covariance matrix of $\hat{\beta}$, which quantifies the variability of model estimates;
* standard errors for each model estimate;
* the proportion of variation captured by a linear model.

Throughout you'll use simple visualizations to help make the connection between fitted models and the aspects of a dataset that model features describe. The lab activity proceeds through the following sections:

0. Exploratory plots for regression analysis
1. Simple linear regression (single explanatory variable)
    + Model fitting
    + Uncertainty quantification
    + Model visualization
2. Multiple linear regression (several explanatory variables)
    + Data preprocessing: encoding categorical variables
    + Interaction terms
    + Model fitting
    + Uncertainty quantification
    + Model visualization

---
### Data: fertility rates and meausres of development 

By way of data, you'll work with country indicators, total fertility rates, and gender indicators for a selection of countries in 2018, and explore the decline in fertility rates associated with developed nations. This has been in the news lately due to preliminary data from the U.S. 2020 census indicating significant [population growth decline in the United States](https://www.washingtonpost.com/dc-md-va/interactive/2021/2020-census-us-population-results/). If the topic interests you, you can read more about perspectives and existing data in this [Our World in Data article](https://ourworldindata.org/fertility-rate). 

The data are stored in separate .csv files imported below:

In [ ]:
fertility = pd.read_csv('data/fertility.csv')
country = pd.read_csv('data/country-indicators.csv')
gender = pd.read_csv('data/gender-data.csv')

The variables you'll work with in this portion are the following:

Dataset | Name | Variable | Units
---|---|---|---
`fertility` | `fertility_total` | National fertility rate | Average number of children per woman
`country` | `hdi` | Human development index | Index between 0 and 1 (0 is lowest, 1 is highest)
`gender` | `edu_expected_yrs_f` | Expected years of education for adult women | Years

Because the variables of interest are stored in three separate dataframes, you'll first need to extract them and merge by country.

In [ ]:
# slice variables of interest
fertility_sub = fertility.loc[:, ['Country', 'fertility_total']]
gender_sub = gender.loc[:, ['educ_expected_yrs_f', 'Country']]
country_sub = country.loc[:, ['Country', 'hdi']]

# merge variables of interest
reg_data = pd.merge(
    fertility_sub, 
    gender_sub, 
    on = 'Country', 
    how = 'inner'
).merge(
    country_sub,
    on = 'Country',
    how = 'left'
).set_index('Country').dropna()

# preview
reg_data.head(4)

---
## 0. Exploratory plots

A preliminary step in regression analysis is typically data exploration through scatterplots. This relies on skills you've already developed -- you may find it helpful to refer to Lab 3 (visualization) throughout this section.

### Q0 (a) Simple scatterplot

Construct a scatterplot of total fertility against expected years of education for women. Label the axes 'Fertility rate' and 'Expected years of education for women'.

(*Remark*: be sure to include `scale = alt.Scale(zero = False)` in the axis specification so that your plot does not have extra whitespace.)

In [ ]:
# solution


The cell below coerces the human development index to a high-med-low factor (categorical variable) using `pd.qcut(...)`. The cutoffs for the factor levels are determined by sample quantiles.

In [ ]:
# labels for the factor
hdi_labels = ['1_low', '2_medium', '3_high']

# cut hdi based on quantiles and store as new variable
reg_data['hdi_fac'] = pd.qcut(reg_data.hdi, 
                              q = 3, 
                              labels = hdi_labels)

# preview
reg_data.head(3)

### Q0 (b) Scatter by HDI level

#### (i) Add HDI level

Modify your plot from Q0 (a) so that points are colored according to the correspondng country's level of human development. Store this plot as `scatter` and display the graphic.

In [ ]:
# solution


#### (ii) Adjust axis scales

Notice that the scatter is clumped together densely in the lower right corner. To better see what's happening in that region, modify your codes for constructing `scatter` so that your axis specifications for both axes include `alt.Scale(zero = False, type = 'pow', exponent = ...)`. 

This will display the axes on a power-transformed scale with an exponent of your choosing. Use an exponent around 2.5 for the education axis, and an exponent around 1/3 for the fertility axis. Store the plot as `scatter_scaled` and display the result.

In [ ]:
# solution


### Q0 (c) Interpret your graphic

Does the negative association (downward trend in the scatter) seem to hold steady among countries at each level of human development? Try to imagine drawing a line to fit the points of each color, and ask yourself whether the slopes would be the same.

(*Hint*: it may help to facet by human development so that you can see the pattern for each development level apart from the other levels.)

#### Answer

*Type your answer here.*

In [ ]:
# facet by hdi level (not required but useful)


### Examining individual points

Often it can be helpful to graphically examine individual points. Especially so if you notice outliers. In this dataset there aren't any apparent outliers, but there are a few points that seem a little farther from the scatter than others, so we can practice on those.

Let's zoom in on the lower-right region of the scatterplot; the cell below applies an Altair filter transform to show only data for which expected education exceeds 12. 

Please be sure you've created `scatter_scaled` in Q0 (b) (ii) before running this cell.

In [ ]:
# zoom in
scatter_highed = scatter_scaled.transform_filter(
    alt.FieldGTPredicate(field = 'educ_expected_yrs_f', gt = 12)
)

scatter_highed

Notice that there are a few points that stand apart from the main scatter. Which countries do  those points represent? It's not to tricky to pick them out by filtering and taking maxima or minima. For example, the country with the lowest fertility rate is: 

In [ ]:
reg_data[reg_data.educ_expected_yrs_f > 12].fertility_total.idxmin()

### Q0 (e) Labeling specific countries

Here you'll look at a few more specific countries, and then label them on the plot.

#### (i) Find the country among the high-HDI nations with the highest total fertility.

In [ ]:
# solution


#### (ii) Find the country among the medium-HDI nations with the highest total fertility.

In [ ]:
# solution


#### (iv) Identify one other country that calls your attention.

In [ ]:
# solution


#### (iv) Modify the cell below to label the three countries identified above. 

In [ ]:
# slice rows for identified countries
point_labels = reg_data.loc[['Singapore']].reset_index() # edit here

# plot text labels
labels = alt.Chart(point_labels).mark_text(align = 'left', dx = 5, dy = 3).encode(
    x = 'educ_expected_yrs_f',
    y = 'fertility_total',
    text = 'Country'
)

# layer over scatter
scatter_highed + labels

---
## 1. Simple linear regression

In this part you'll fit a simple linear model regressing fertility on education.

First we'll need to store the quantities -- the response and explanatory variables -- needed for model fitting in the proper format. Recall that the linear model in matrix form is:

$$\underbrace{\left[\begin{array}{c} y_1 \\ \vdots \\ y_n \end{array}\right]}_{\mathbf{y}}
    = \underbrace{\left[\begin{array}{cc}
        1 & x_1 \\ \vdots &\vdots \\ 1 & x_n
        \end{array}\right]}_\mathbf{X}
      \underbrace{\left[\begin{array}{c} \beta_0 \\ \beta_1 \end{array}\right]}_\beta
      + \underbrace{\left[\begin{array}{c} \epsilon_1 \\ \vdots \\ \epsilon_n \end{array}\right]}_\epsilon$$
      
Notice that the explanatory variable matrix $\mathbf{X}$ includes a column of ones for the intercept. So the quantities needed are:
* $\mathbf{y}$, a one-dimensional array of the total fertility rates for each country; and 
* $\mathbf{X}$, a two-dimensional array with a column of ones (intercept) and a column of the expected years of education for women (explanatory variable).

The cell below prepares the explanatory variable matrix $\mathbf{X}$ and response vector $\mathbf{y}$ in the following steps:

1. Store the total fertility rates as an array named `y`.
2. Slice the `educ_expected_yrs_f` variable from the `reg_data` dataframe, resulting in a dataframe with one column stored as `x_slr_df`.
3. Append a column of ones to `x_slr_df` using `add_dummy_feature`, resulting in a 139-by-2 array stored as `x_slr`.

In [ ]:
# store response variable as array
y = reg_data.fertility_total

# store explanatory variable
x_slr_df = reg_data.loc[:, ['educ_expected_yrs_f']]

# add intercept column
x_slr = add_dummy_feature(x_slr_df, value = 1)

# print first five rows
x_slr[0:5, :]

### Model fitting

'Fitting' a model refers to computing estimates; it is typical practice to report estimated parameters and standard errors, but first we'll focus on computing the estimates. This is done using the function `LinearRegression()` in the `sklearn.linear_model` module. The syntax is superficially somewhat similar to using `PCA(...)` in the previous lab (also from an `sklearn` module): we'll configure `LinearRegression(...)` as `slr`, and then fit the model using the `fit(...)` method.

In [ ]:
# configure module
slr = LinearRegression(fit_intercept = False)

# fit model
slr.fit(x_slr, y)

Notice the argument `fit_intercept = False` -- this configures `slr` *not* to fit an intercept, which is done here since the explanatory variable matrix already includes a column of ones for the intercept term.

### Q1 (a) Retrieve estimated coefficients

After the `fit(...)` method is applied, the coefficient estimates are stored as the `.coef_` attribute of `slr`. 

#### (i) Print the coefficient estimates.

In [ ]:
# print coefficient estiamtes


The order in which the `.coef_` attribute stores the estimates matches the order of the columns of $\mathbf{X}$.

#### (ii) Which is which?

Identify the intercept and identify the slope. Replace the dots below.

* Intercept estimate: $\hat{\beta}_0 = \dots$
* Slope estimate: $\hat{\beta}_1 = \dots$

(*Hint*: refer back to the plot; only one of the two estimates could possibly be the slope.)

#### (iii) Interpret the slope coefficient.

Fill in the blanks below.

Among the countries in the sample, a one-year increase in ___________________ is associated with a ____________ in the average number of children per woman.

### Fitted values and residuals

The 'fitted value' for $y_i$ is the value along the line specified by the model that corresponds to the matching explanatory variable $x_i$. In other words:

$$\hat{y}_i = \hat{\beta}_0 + \hat{\beta}_1 x_i$$

These can be obtained by passing the explanatory variable matrix to the `.predict(...)` method. In effect, this returns predictions on the observed data.

In [ ]:
# fitted values
fitted_slr = slr.predict(x_slr)

fitted_slr[0:5]

The result is an array with length matching the number of rows in `x_slr`. 

In [ ]:
# dimensions
fitted_slr.shape

### Q1 (b) Compute the residuals

The 'residuals' -- what's left over -- are the differences between the fitted and observed values of the response:

$$e_i = y_i - \hat{y}_i \qquad\text{$i$th residual}$$

Use `fitted_slr` and `y` to compute an array of the residuals. Store the result as `resid_slr`.

In [ ]:
# residuals


### Q1 (c) Append fitted values and residuals to the data

In order to keep them handy, add the fitted values and residuals as new columns in `reg_data` with the names `fitted_slr` and `resid_slr`. Print the first few rows of `reg_data` after adding the new variables.

In [ ]:
# append

# print


There's one more estimate to compute: the error variance, $\sigma^2$! This can be estimated based on the residual variance:

$$\hat{\sigma}^2 = \frac{1}{n - p}\sum_{i = 1}^n e_i^2 = \frac{n - 1}{n - p} S^2_e$$

In this expression, $e_i$ are the residuals, $n$ and $p$ are the dimensions of $\mathbf{X}$, and $S^2_e$ is the residual (sample) variance.

In [ ]:
# store n (number of observations) and p (number of betas)
n, p = x_slr.shape

# compute estimate of error variance
sigma2_hat = ((n - 1)/(n - p)) * resid_slr.var()

### Uncertainty quantification

It was noted in lecture that the variances and covariances of $\hat{\beta}_0, \hat{\beta}_1$ are given by the matrix:

$$\sigma^2 (\mathbf{X}'\mathbf{X})^{-1} 
  = \left[\begin{array}{cc} 
        \text{var}\hat{\beta}_0 & \text{cov}\left(\hat{\beta}_0, \hat{\beta}_1\right) \\
        \text{cov}\left(\hat{\beta}_1, \hat{\beta}_0\right) & \text{var}\hat{\beta}_1
        \end{array}\right]$$

So we can *estimate* these quantities, which quantify the variation and covariation of the estimated coefficients, by plugging in the estimated error variance and computing:

$$\hat{\sigma}^2 (\mathbf{X}'\mathbf{X})^{-1}$$

In [ ]:
# compute X'X
xtx_slr = x_slr.transpose().dot(x_slr)

# compute matrix of parameter variances/covariances: estimated error variance x (X'X)^{-1}
slrcoef_vcov = np.linalg.inv(xtx_slr) * sigma2_hat

# print
slrcoef_vcov

### Q1 (d) Standard errors

The *standard errors* for the estimates $\hat{\beta}_0, \hat{\beta}_1$ are the square roots of their estimated variances. Compute these by retrieving the diagonal elements of `slrcoef_vcov` (the estimated variances) and taking square roots.

Store the result (an array with two values) as `slrcoef_se` and print the array.

(*Hint*: use `.diagonal()` from numpy.)

In [ ]:
# take square root of matrix diagonals to get standard errors 

# print


We can now report the results of model fitting in an organized fasion: we'll make a dataframe `slrcoef_table` with two columns, 'coefficient estimate' and 'standard error', and index the rows of the dataframe by the coefficient names (intercept and education). 

In [ ]:
# create table
slrcoef_table = pd.DataFrame(
    data = {'coefficient estimate': slr.coef_, 'standard error': slrcoef_se},
    index= ['intercept', 'education']
)

# print
slrcoef_table

Lastly, a standard metric often reported with linear models is the $R^2$ score, which is interpreted as the proportion of variation in the response captured by the model. It is straightforward to compute using the `r2_score(...)` function from `sklearn.metrics`:

In [ ]:
# compute R-squared
r2_score(reg_data.fertility_total, reg_data.fitted_slr)

But just so you have a sense of what it is, a direct calculation is shown below:

In [ ]:
# R-squared 'by hand'
(y.var() - resid_slr.var())/y.var()

The metric is simply the difference between the raw variation in the response and residual variation, as a proportion of the variation in the response. If the model fits well, the residual variation will be small, in which case this proportion will be closer to 1.

### Q1 (e) Visualize!

Now that you've reported the numerical results of model fitting, have a direct look at the results relative to the data scatter.

#### (i) Construct a line plot of the fitted values against expected years of education.

Remember that the fitted values are stored as `fitted_slr` in the dataframe `reg_data` containing the actual data. 

Construct the line as `slr_line` and layer this on top of your plot `simple_scatter` from part 0 above.

In [ ]:
# construct line plot
slr_line = ...

# layer


#### (ii) Comment on the plot.

How well does the model seem to describe the data overall?

#### Answer

Pretty well -- the line seems to go straight through the scatter and captures the overall trend nicely.

---
## 2. Multiple linear regression

Now let's consider adding the human development factor -- a categorical variable -- to the model. The factor column `hdi_fac` can't be input directly as a variable into the explanatory variable matrix $\mathbf{X}$, because it doesn't make sense to multiply a coefficient (number) by a factor label (character string), so we'll need to do a little preprocessing to incorporate the factor into a multiple regression model.

### Encoding categorical variables

Categorical variables must be *encoded* in the explanatory variable matrix by *indicator functions*; each level of the category will have a corresponding coefficient. In this case, the factor (HDI) had three levels (low, medium, and high), so we'll construct a matrix $\mathbf{X}$ with entries

$$\left[\begin{array}{cccc}
    1 &x_1 &I(hdi_1 = med) &I(hdi_1 = high) \\
    1 &x_2 &I(hdi_2 = med) &I(hdi_2 = high) \\
    \vdots &\vdots &\vdots &\vdots \\
    1 &x_n &I(hdi_n = med) &I(hdi_n = high)
    \end{array}\right]$$
    
where
* $I(hdi_i = med)$ is 1 if the $i$th HDI level is medium, and 0 otherwise and
* $I(hdi_i = high)$ is 1 if the $i$th HDI level is high, and 0 otherwise.

The low level of the HDI factor is not represented explicitly, but if $I(hdi_i = med) = I(hdi_i = high) = 0$, then $hdi_i = low$; in other words, if the $i$th level of the HDI factor is neither medium nor high, then it must be low. 

It takes a little head-scratching to figure out what these indicators do, but **their effect in the model is to allow the intercept to change depending on the level of the factor**.

These indicators functions are sometimes called 'dummy' variables. They're simple to obtain using `pd.get_dummies(...)`:

In [ ]:
# encode hdi factor by indicator variables
hdi_df = pd.get_dummies(reg_data.hdi_fac, drop_first = True)

# preview
hdi_df.head(6)

Now compare this with the underlying factor:

In [ ]:
reg_data[['hdi_fac']].head(6)

### Interaction terms

We'll also add *interaction* terms -- a fancy phrase for products of explanatory variables -- between the indicators and the education variable. **The effect of interactions is to allow the slope to change depending on the level of the factor**. 

The cell below computes these interaction terms and appends them, along with the indicators, to the education variable.

In [ ]:
# compute interaction terms
interaction_df = hdi_df.multiply(reg_data.educ_expected_yrs_f, axis = 0)
interaction_df.columns = ['2_medium_x_educ', '3_high_x_educ']

# append indicators to data
x_mlr_df = pd.concat([reg_data.educ_expected_yrs_f, hdi_df, interaction_df], ignore_index = False, axis = 1)

# preview
x_mlr_df.head(4)

Notice that those interaction terms are simply the elementwise products of each indicator column with the education variable column.

### Model fitting

With the encoded categorical variable for HDI, and the interaction terms, the linear model is:

$$\underbrace{\left[\begin{array}{c} y_1 \\ \vdots \\ y_n \end{array}\right]}_{\mathbf{y}}
    = \underbrace{\left[\begin{array}{cccccc}
        1 & x_1 & I(hdi_1 = med) & I(hdi_1 = high) & x_1 I(hdi_1 = med) & x_1 I(hdi_1 = high) \\ 
        \vdots &\vdots &\vdots &\vdots &\vdots &\vdots \\ 
        1 & x_n & I(hdi_n = med) & I(hdi_n = high) & x_n I(hdi_n = med) & x_n I(hdi_n = high)  
        \end{array}\right]}_\mathbf{X}
      \underbrace{\left[\begin{array}{c} \beta_0 \\ \beta_1 \\ \beta_2\\ \beta_3 \\ \beta_4 \\ \beta_5 \end{array}\right]}_\beta
      + \underbrace{\left[\begin{array}{c} \epsilon_1 \\ \vdots \\ \epsilon_n \end{array}\right]}_\epsilon$$
      
Which is matrix form for:

$$y_i = \beta_0 + \beta_1 x_1 + \beta_2 I(hdi_1 = med) + \beta_3 I(hdi_1 = high) + \beta_4 x_1 I(hdi_1 = med) + \beta_5 x_1 I(hdi_1 = high) + \epsilon_i \qquad i = 1, \dots, n$$

While seemingly more complicated, and perhaps a little tricker to interpret mathematically, the syntactical pattern for fitting the model is exactly the same as the simple linear model.

In [ ]:
# add intercept column
x_mlr = add_dummy_feature(x_mlr_df, value = 1)

# configure module
mlr = LinearRegression(fit_intercept = False)

# fit model
mlr.fit(x_mlr, y)

### Q2 (a) Summarize model fit

Follow the examples from the simple linear model to carry out the following:

1. Compute the error variance estimate $\hat{\sigma}^2$.
2. Compute the standard errors for each coefficient.
3. Construct a dataframe showing both the estimates and standard errors.

(*Suggestion*: copy the codes from part 1 into a single cell below for reference.)

In [ ]:
# store dimensions of explanatory variable matrix

# compute residual variance

# compute standard errors

# construct coefficient table

# print


### Q2 (b) Fit metric

Now let's compare the fit of the MLR model to that of the SLR model.

#### (i) Calculate the $R^2$ score for the multiple linear regression model.


In [ ]:
# solution


#### (ii) Does the fit seem to have improved appreciably?

#### Answer

*Type your answer here.*

### Q2 (c) Visualize

Now visualize the model outputs. 

1. First append the fitted values (if you didn't above) as a new column named `fitted_mlr` to `reg_data`.
2. Construct a line plot of the fitted values (y axis) against education (x axis) and map the HDI factor to the color aesthetic. This should produce a plot with three lines, one for each level of HDI. Store the result as `mlr_lines`.
3. Layer `mlr_lines` on top of `scatter_scaled`.

In [ ]:
# append fitted values to original data

# construct line plot

# layer


### Q2 (d) Interpret your graphic

What does the multiple linear model suggest about the relationship between fertility rate and expected years of education for high-HDI countries as compared with low-HDI countries?

(*Hint*: focus on how the slopes of the two lines compare.)

#### Answer

*Type your answer here.*

### Comment

These data are definitely *not* a representative sample of any particular population of nations -- the countries (observational units) are conveniently chosen based on which countries reported data. So there is no scope of inference here.

Although we can't claim that, for example, 'the mean fertility rate decreases with education at a rate of -0.35 children per woman per expected year of education in low-HDI countries', we *can* say '*among the countries reporting data*, the mean fertility rate decreases with education at a rate of -0.35 children per woman per expected year of education in low-HDI countries'. This is a nice example of how a model might be used in a descriptive capacity.

---
## Submission Checklist
1. Save file to confirm all changes are on disk
2. Run *Kernel > Restart & Run All* to execute all code from top to bottom
3. Save file again to write any new output to disk
4. Select *File > Download as > HTML*.
5. Open in Google Chrome and print to PDF on A3 paper in portrait orientation.
6. Submit to Gradescope